# Processing 

In [2]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
import json

rep_src   = "data/raw/"       # Fichiers téléchargés avant traitement
rep_inter = "data/inter/"     # Des fichiers intermédiaires sont utilisés
rep_dst   = "data/processed/" # Fichiers utilisés par la modélisation

# La liste 'actions' sert à stocker la liste des actions réalisées.
# Elle est affichée en fin de notebook et sert à la rédaction du rapport
# La fonction log_action affiche et stocke dans actions[].
actions  = []
def log_action(str):
    print (f" - {str}")
    actions.append(str)

# Il y a des variables à écarter, nous enregistrons la liste dans var_ecartees
# Nous enregistrons pour chaque var. écartée son nom et la raison.
var_ecartees = []
    
# Lecture de la liste des fichiers, cette liste comprend : les noms
# des fichiers, les séparateurs, la période ("phase"), les conversions
# de type à réaliser et quelques autre infos.
with open("./desc_fic_raw.json", 'r', encoding='utf-8') as fichier:
    des_fic_raw = json.load(fichier)

# Lecture de la description des variables avec leurs libellés
# et les correspondances des modalités pour des affichages explicites
with open("./desc_vars.json", 'r', encoding='utf-8') as fichier:
    desc_vars = json.load(fichier)

dfrub = {}

# Lecture des données

Les données sont réparties dans 4 rubriques et dans 4 années de 2019 à 2022

Les informations sur les fichiers contenues dans le fichier desc_fic_raw.json
contiennent :
  - Le nom du fichier ;
  - Le séparateur ;
  - La phase : jusqu'à 2018 ou à partir de 2019 ;
  - les conversions de types dans dtypes à réaliser lors du chargement ;
 Le code de la cellule suivante réalise :
  - la lecture des fichiers ;
  - des conversions de type et le remplacement des " -1" par "-1"
  - le renommage d'une colonne d'un fichier ;
  - la concaténation des fichiers de chaque rubrique

In [4]:
#!pwd
#!ls -l data/raw

# Traitement  : remplacement de " -1" par "-1"
for rubrique, description_rub in des_fic_raw.items():  # Pour chaque rubrique
    nb_obs = 0  # nombre total d'observation (lignes, hors entêtes)

    print ()
    print("Rubrique : ", rubrique)
    
    dfl = []
    dtype = description_rub.get("dtypes")
    

    for fichier_origine in description_rub["fichiers"]:  # Pour chaque fichier annuel
        if fichier_origine.get("phase") == "2":
            nom_fichier = fichier_origine["nom"]
            # lecture du fichier
            df = pd.read_csv(rep_src + nom_fichier,
                             sep=fichier_origine["sep"],
                             dtype=dtype,
                             encoding="latin_1",
                             index_col=False,
                             quotechar="\"",
                             low_memory=False)
            nb_obs += df.shape[0]
            if fichier_origine.get("rename_cols") is not None:
                df = df.rename(columns=fichier_origine.get("rename_cols"))
                print("  - rename ", fichier_origine.get("rename_cols"))
            print(nom_fichier, df.shape)  # Pour info et mise au point
            dfl.append(df)

    dfrub[rubrique] = pd.concat(dfl) # Concaténation de tous les DataFrames annuels (2019->2022)
    
    # Remplacement des " -1" par "-1"
    # Les valeurs manquantes sont codées "-1" et parfois " -1", avec une espace en plus
    # Il faut alors supprimer cette espace qui n'a pas de sens.
    dfrub[rubrique] = dfrub[rubrique].replace(" -1", "-1")

    print("Nombre de DataFrames  : ", len(dfl))
    print("Nombre d'observations : ", nb_obs, dfrub[rubrique].shape[0])
    print("Colonnes du DataFrame : ", dfrub[rubrique].shape[1])

    dfrub[rubrique].to_csv(rep_dst + rubrique + ".csv", sep='\t', index=False)

dfu = dfrub["usagers"]
dfv = dfrub["vehicules"]
dfl = dfrub["lieux"]
dfc = dfrub["caracteristiques"]


Rubrique :  caracteristiques
caracteristiques-2019.csv (58840, 15)
caracteristiques-2020.csv (47744, 15)
carcteristiques-2021.csv (56518, 15)
  - rename  {'Accident_Id': 'Num_Acc'}
carcteristiques-2022.csv (55302, 15)
Nombre de DataFrames  :  4
Nombre d'observations :  218404 218404
Colonnes du DataFrame :  15

Rubrique :  usagers
usagers-2019.csv (132977, 15)
usagers-2020.csv (105295, 15)
usagers-2021.csv (129248, 16)
usagers-2022.csv (126662, 16)
Nombre de DataFrames  :  4
Nombre d'observations :  494182 494182
Colonnes du DataFrame :  16

Rubrique :  vehicules
vehicules-2019.csv (100710, 11)
vehicules-2020.csv (81066, 11)
vehicules-2021.csv (97315, 11)
vehicules-2022.csv (94493, 11)
Nombre de DataFrames  :  4
Nombre d'observations :  373584 373584
Colonnes du DataFrame :  11

Rubrique :  lieux
lieux-2019.csv (58840, 18)
lieux-2020.csv (47744, 18)
lieux-2021.csv (56518, 18)
lieux-2022.csv (55302, 18)
Nombre de DataFrames  :  4
Nombre d'observations :  218404 218404
Colonnes du DataF

## Enregistrement des 4 rubriques de 2019 à 2022
Ces enregistrements servent à l'exploration et à la mise au point. Il sont provisoires. Ce code est commenté en temps normal.

In [6]:
dfc.to_csv (rep_inter+'caracteristiques_raw_4.csv',   sep = '\t')
dfl.to_csv (rep_inter+'lieux_raw_4.csv',              sep = '\t')
dfu.to_csv (rep_inter+'usagers_raw_4.csv',            sep = '\t')
dfv.to_csv (rep_inter+'vehicules_raw_4.csv',          sep = '\t')

# Jointure des 4 rubriques

Les jointures sont toutes faites avec le champ Num_Acc
Le type de Num_Acc est forcé à int lors de la lecture par read_csv()
La dernière jointure avec les véhicules est faite avec, en plus,
les champs num_veh et id_vehicule.
Les jointures sont "à gauche" ("left")
pour conserver le nombre d'usagers.
Les nombres d'observations et de variables affichés avant et après
chaque jointure permettent de vérifier les jointures.
Il y a 494 182 usagers avant les jointures et le DataFrame résultant
a ce même nombre d'observations.


In [8]:
########################################################################
# Jointures des 4 rubriques 
########################################################################

print("")
log_action("Jointure usagers  <---  caracteristiques")
#print("Un pour un")
print ()
print("Taille usagers          : ", dfu.shape)
print("Taille caractéristiques : ", dfc.shape)
df = pd.merge(dfu, dfc, on="Num_Acc", how = "left")
print("Tailles df résultant    : ", df.shape)

print("")
log_action("Jointure (usagers et caracteristiques)  <---  lieux")
print()
print("Taille DataFrame        : ", df.shape)
print("Taille lieux            : ", dfl.shape)
df = pd.merge(df, dfl, on="Num_Acc", how = "left")
print("Tailles df résultant    : ", df.shape)

print("")
log_action("Jointure (usagers, caracteristiques et lieux)  <---  vehicules")
print()
print("Taille DataFrame        : ", df.shape)
print("Taille vehicules        : ", dfv.shape)
df = pd.merge(df, dfv, on=["Num_Acc", "id_vehicule", "num_veh"], how = "left")

print("Tailles df résultant    : ", df.shape)
print ()
print ("Colonnes résultantes : ", df.columns)

print (df.info(max_cols=1000, show_counts=True))
# Libérons de la mémoire.
# Les DataFrames des 4 rubriques ne seront plus utilisés
dfc = None
dfl = None
dfu = None
dfv = None
dfrub = None


 - Jointure usagers  <---  caracteristiques

Taille usagers          :  (494182, 16)
Taille caractéristiques :  (218404, 15)
Tailles df résultant    :  (494182, 30)

 - Jointure (usagers et caracteristiques)  <---  lieux

Taille DataFrame        :  (494182, 30)
Taille lieux            :  (218404, 18)
Tailles df résultant    :  (494182, 47)

 - Jointure (usagers, caracteristiques et lieux)  <---  vehicules

Taille DataFrame        :  (494182, 47)
Taille vehicules        :  (373584, 11)
Tailles df résultant    :  (494182, 55)

Colonnes résultantes :  Index(['Num_Acc', 'id_vehicule', 'num_veh', 'place', 'catu', 'grav', 'sexe',
       'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp',
       'id_usager', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg',
       'int', 'atm', 'col', 'adr', 'lat', 'long', 'catr', 'voie', 'v1', 'v2',
       'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout',
       'surf', 'infra', 'situ', 'vma', 'senc', 'cat

# Suppression d'observations
Nous supprimons les observations dont la gravité est inconnue,
la gravité inconnue est codée -1

In [10]:
nb_avant = df.shape[0]
print (f"Nombre d'observations avant suppression {nb_avant}")
df1 = df.loc[df.grav!='-1',:]
print (f"Nombre d'observations supprimées        {nb_avant- df.shape[0]}")
print (f"Nombre d'observations après suppression {df.shape[0]}")
log_action(f"Suppression de {nb_avant- df.shape[0]} observations dont la gravité est inconnue (codée -1)")

Nombre d'observations avant suppression 494182
Nombre d'observations supprimées        0
Nombre d'observations après suppression 494182
 - Suppression de 0 observations dont la gravité est inconnue (codée -1)


# Création de variables

## Création de la variable indiquant un jour ferié

Cette variable nous permettra de déterminer si les jours feriés
expliquent la gravité des accidents. Les jours fériés sont les dimanches
et les jours de fête.

## Création de la variable du jour de la semaine

Cette variable nous permettra de déterminer si le jour de la
semaine explique la gravité des accidents.

## Création de la variable indiquant l'âge

Le jeu de données contient l'année de naissance des usagers et l'année
de l'accident. Nous créons la variable age égal à la différence
entre l'année de l'accident et l'année de naissance.
Lorsque l'année de naissance est inconnue l'âge est codé -1

In [12]:
# Creation de colonnes :
#   - ferie : 0 jour ouvré, 1 : jour ferié
#   - jsem : jour de la semaine : lundi : 1 , ... , 7 : dimanche

##############################################################################
# Inclusion du jour de la semaine
##############################################################################

print("  - Inclusion du jour de la semaine dans le fichier")
df_date = df[["an", "mois", "jour"]]
df_date = df_date.rename({"an": "year", "mois": "month", "jour": "day"}, axis=1)
df_date["ts"] = pd.to_datetime(df_date)
# weekday() return :  Monday is 0 and Sunday is 6.
# La modalité zéro signifiant parfois non renseigné ou non applicable
# nous préférons ajouter 1 à la valeur retournée par weekday()
df_date["jsem"] = df_date.ts.apply(lambda x: x.weekday()+1)

df["jsem"] = df_date.jsem
df_date = None  # Libérons un peu de mémoire

print ("Jours de la semaine : ")
print (df.jsem.value_counts().sort_index())
log_action (f"Création de la variable jsem : jour de la semaine")

##############################################################################
# Création de la variable ferie
# elle vaut 1 si le jour est ferie, 0 sinon
# Les jours fériés sont les dimanches et les jours de fête.
##############################################################################

df["ferie"] = False

# Les dimanches
df.loc[df.jsem==7,"ferie"] = True

# Les 1er janvier
df.loc[(df.mois == '1') & (df.jour == '1'), 'ferie'] = True

# Les 1er mai
df.loc[(df["mois"]==5) & (df["jour"]==1), "ferie"] = True

# Les 8 mai
df.loc[(df.mois==5) & (df.jour==8), "ferie"] = True

# Les 14 juillet
df.loc[(df.mois==7) & (df.jour==14), "ferie"] = True

# Les 15 août
df.loc[(df.mois==7) & (df.jour==14), "ferie"] = True

# Les 1er novembre
df.loc[(df.mois==11) & (df.jour==1), "ferie"] = True

# Les 11 novembre
df.loc[(df.mois==11) & (df.jour==11), "ferie"] = True

# Les 25 décembre
df.loc[(df.mois==12) & (df.jour==25), "ferie"] = True

# Dimanche de pâques, date variant chaque année
df.loc[(df.an == 2019) & (df.mois==4) & (df.jour==21), "ferie"] = True
df.loc[(df.an == 2020) & (df.mois==4) & (df.jour==12), "ferie"] = True
df.loc[(df.an == 2021) & (df.mois==4) & (df.jour==4),  "ferie"] = True
df.loc[(df.an == 2022) & (df.mois==4) & (df.jour==17), "ferie"] = True

# Lundi de pentecôte, date variant chaque année
df.loc[(df.an == 2019) & (df.mois==6) & (df.jour==10), "ferie"] = True
df.loc[(df.an == 2020) & (df.mois==6) & (df.jour==1), "ferie"] = True
df.loc[(df.an == 2021) & (df.mois==5) & (df.jour==24),  "ferie"] = True
df.loc[(df.an == 2022) & (df.mois==6) & (df.jour==6), "ferie"] = True

# Jeudi de l'ascension, date variant chaque année
df.loc[(df.an == 2019) & (df.mois==5) & (df.jour==30), "ferie"] = True
df.loc[(df.an == 2020) & (df.mois==5) & (df.jour==21), "ferie"] = True
df.loc[(df.an == 2021) & (df.mois==5) & (df.jour==13),  "ferie"] = True
df.loc[(df.an == 2022) & (df.mois==5) & (df.jour==26), "ferie"] = True

# Saint Étienne en alsace Moselle, le 26 décembre
df.loc[ df.dep.isin([57, 67, 68]) & (df.mois==12) & (df.jour==26), "ferie"] = True

# Abolition de l'esclavage en Guadeloupe
df.loc[(df.dep == 971) & (df.mois==5) & (df.jour==27), "ferie"] = True

# Abolition de l'esclavage en Martinique
df.loc[(df.dep == 972) & (df.mois==5) & (df.jour==22), "ferie"] = True

# Abolition de l'esclavage en Guyane
df.loc[(df.dep == 973) & (df.mois==6) & (df.jour==10), "ferie"] = True

# Abolition de l'esclavage à la Réunion
df.loc[(df.dep == 974) & (df.mois==12) & (df.jour==20), "ferie"] = True

# Abolition de l'esclavage à Mayotte
df.loc[(df.dep == 976) & (df.mois==4) & (df.jour==27), "ferie"] = True

# Abolition de l'esclavage à Saint-Barthélémy
df.loc[(df.dep == 977) & (df.mois==10) & (df.jour==9), "ferie"] = True

# Abolition de l'esclavage à Saint-Martin
df.loc[(df.dep == 978) & (df.mois==5) & (df.jour==28), "ferie"] = True

print ("Jours ouvrés et fériés")
print (df.ferie.value_counts())
log_action (f"Création de la variable ferie : jour férié, dimanches et autres fêtes")


##############################################################################
# Création de la variable age
# Elle est égale à l'année de l'accident moins l'année de naissance à un an près.
# L'année de naissance n'est pas toujours connue, elle est alors codée -1
# TODO : gérer l'arrondi
##############################################################################

print (df.an_nais.isna().sum())

# Lorsque la date de naissance est inconnue,
# l'âge est codé avec une valeur négative.
ag = df.an.astype("int") - df.an_nais.fillna(2030).astype ("int")
ag.loc[ag<0] = -1
df["age"] = ag
ag = None  # Libération de la mémoire

log_action (f"Création de la variable age : différence entre l'année de l'accident et l'année de naissance")

  - Inclusion du jour de la semaine dans le fichier
Jours de la semaine : 
jsem
1    66099
2    70041
3    70392
4    71101
5    81515
6    72974
7    62060
Name: count, dtype: int64
 - Création de la variable jsem : jour de la semaine
Jours ouvrés et fériés
ferie
False    425236
True      68946
Name: count, dtype: int64
 - Création de la variable ferie : jour férié, dimanches et autres fêtes
5941
 - Création de la variable age : différence entre l'année de l'accident et l'année de naissance


# Dichotomisation  / catégorisation
La plupart des variables du jeu de données sont qualitatives et
le modèle entraîné doit faire de la catégorisation.
Nous procédons alors à la dichotomisation des variables catégorielles.
Nous avons constaté la présence de valeurs non renseignées dans le jeu de données,
elles sont parfois absentes et le plus souvent codées "-1" ou " -1".
Nous avons aussi constaté dans certaines variables la modalité "non applicable",
c'est par exemple le cas de la place de l'usager dans le véhicule lorsqu'il est piéton.
Nous utilisons une fonction 


In [14]:
##############################################################################
# Fonction de dichotomisation adaptée
##############################################################################

def dichotomisation (df, column, var_ecartees, desc_vars=None, dummies=None, mod_ecartees = None):
    """
    Cette fonction fait la dichotomisation des seules modalités de la
    liste fournie par dummies. Elle permet de ne pas dichotomiser les
    modalités : "non renseigné", "Autre", "Non applicable", ...
    Elle utilise si possible les infos de desc_vars pour nommer les colonnes.
    et elle complète desc_vars.
    """
    try :
        desc_vars = desc_vars.get("columns")
        col_desc = {}
        for c in desc_vars :
            if c.get("name") == column:
                col_desc = c
                break
    except :
        col_desc = {}

    if dummies is None:
        dum = list(df[column].unique())
    else:
        dum = dummies
    if mod_ecartees is not None:
        dum = [x for x in dum if x not in mod_ecartees]
    print()
    
    # Pour chaque variable correspondant à une modalité
    for c in dum: # c : modalité
        
        # Le nom de la nvle variable est le nom de la var. et la modalité
        new_col_name = column + "_" + str(c)
        
        # Création de la nouvelle variable (colonne)
        df[new_col_name] = df[column] == c
        
        # Recherche d'une description existante dans la liste des descriptions
        # les descriptions sont dans un tableau qu'il faut adresser avec un entier
        desc_new_col = None
        for ic in range(len(desc_vars)):
            if desc_vars[ic].get("name") == new_col_name:
                desc_new_col = desc_vars[ic]
                break
            
        if desc_new_col is None:
            desc_new_col = {}
            desc_new_col["name"] = new_col_name
            desc_new_col["dtype"] = "bool"
            values = col_desc.get("values")
            if values is not None and values.get(c) is not None:
                desc_new_col["label"] = col_desc.get("label") + " : " + values.get(c)
            else :
                desc_new_col["label"] = col_desc.get("label")
            desc_vars.append(desc_new_col)

    var_ecartees.append ((column, "Dichotomisation"))
    # df = df.copy()
    return

In [15]:
##############################################################################
# Dichotomisation des champs secu 1 à 3
##############################################################################
# L'usager peut être protégé par plusieurs équipements, il y a trois variables
# pour décrire ces équipements, il y a 6 types d'équipement, il faut alors
# créer six modalités. 

df["secu_ceinture"]   = False
df["secu_casque"]     = False
df["secu_dispenfant"] = False
df["secu_gilet"]      = False
df["secu_airbag23RM"] = False
df["secu_gants"]      = False

df.loc[df.secu1 == '1', "secu_ceinture"] = True
df.loc[df.secu2 == '1', "secu_ceinture"] = True
df.loc[df.secu3 == '1', "secu_ceinture"] = True

df.loc[df.secu1 == '2', "secu_casque"] = True
df.loc[df.secu2 == '2', "secu_casque"] = True
df.loc[df.secu3 == '2', "secu_casque"] = True

df.loc[df.secu1 == '3', "secu_dispenfant"] = True
df.loc[df.secu2 == '3', "secu_dispenfant"] = True
df.loc[df.secu3 == '3', "secu_dispenfant"] = True

df.loc[df.secu1 == '4', "secu_gilet"] = True
df.loc[df.secu2 == '4', "secu_gilet"] = True
df.loc[df.secu3 == '4', "secu_gilet"] = True

df.loc[df.secu1 == '5', "secu_airbag23RM"] = True
df.loc[df.secu2 == '5', "secu_airbag23RM"] = True
df.loc[df.secu3 == '5', "secu_airbag23RM"] = True
df.loc[df.secu1 == '7', "secu_airbag23RM"] = True
df.loc[df.secu2 == '7', "secu_airbag23RM"] = True
df.loc[df.secu3 == '7', "secu_airbag23RM"] = True

df.loc[df.secu1 == '6', "secu_gants"] = True
df.loc[df.secu2 == '6', "secu_gants"] = True
df.loc[df.secu3 == '6', "secu_gants"] = True
df.loc[df.secu1 == '7', "secu_gants"] = True
df.loc[df.secu2 == '7', "secu_gants"] = True
df.loc[df.secu3 == '7', "secu_gants"] = True

var_ecartees.append(("secu1", "Dichotomisation"))
var_ecartees.append(("secu2", "Dichotomisation"))
var_ecartees.append(("secu3", "Dichotomisation"))
#df = df.drop(["secu1", "secu2", "secu3"], axis = 1)
log_action(f"Dichotomisation des champs secu1, secu2 et secu3")

##############################################################################
# Dichotomisation de l'âge
# TODO : À affiner
##############################################################################
df["age_enfant"]  = (df.age>=0) & (df.age<= 15)
df["age_jeune"]   = (df.age>15) & (df.age<= 25)
df["age_adulte"]  = (df.age>25) & (df.age<= 64)
df["age_3age"]    = (df.age>64)

var_ecartees.append(("age", "Dichotomisation"))
log_action(f"Dichotomisation de l'âge")

##############################################################################
# Dichotomisation de l'heure
##############################################################################
# TODO : faire un équilibrage
df["hr_matin"] = (df.hrmn>="0600") & (df.hrmn<"1200") 
df["hr_midi"]  = (df.hrmn>="1200") & (df.hrmn<"1400") 
df["hr_am"]    = (df.hrmn>="1400") & (df.hrmn<"1800") 
df["hr_soir"]  = (df.hrmn>="1800") & (df.hrmn<"2100") 
df["hr_nuit"]  = (df.hrmn>="2100") | (df.hrmn<"0600") 

var_ecartees.append(("hrmn", "Dichotomisation"))
log_action(f"Dichotomisation de l'heure")

##############################################################################
# Dichotomisation du sexe
##############################################################################
df["sexe_m"] = df.sexe == '1'
df["sexe_f"] = df.sexe == '2'

var_ecartees.append(("sexe", "Dichotomisation"))
log_action(f"Dichotomisation du sexe")

##############################################################################
# Dichotomisation de la gravité
##############################################################################
# Les noms de variables seront plus faciles à retenir que les valeurs 1 à 4
# N.B.: les observations dont la gravité est inconnue (codée -1) seront écartées
df["grav_grave"]       = df.grav.isin(["2", "3"])

var_ecartees.append(("grav", "Dichotomisation"))
log_action(f"Dichotomisation de la gravité")

##############################################################################
# Dichotomisation du nombre de voies de circulation avec regroupement
##############################################################################
df["nbv_1"]    = df.nbv == '1'
df["nbv_2"]    = df.nbv == '2'
df["nbv_3"]    = df.nbv == '3'
df["nbv_4"]    = df.nbv == '4'
df["nbv_plus"] = df.nbv.isin(['5', '6', '7', '8', '9', '10','11', '12'])

var_ecartees.append(("nbv", "Dichotomisation"))
log_action(f"Dichotomisation du nombre de voies avec regroupement 1 à 4 puis 5 et plus")

##############################################################################
# Dichotomisation de l'état de la surface
##############################################################################
df["surf_norm"]  = df.surf == '1'
df["surf_mouil"] = df.surf == '2'
df["surf_gliss"] = df.surf.isin(['3', '4', '5', '6', '7', '8', '9'])
df["surf_autre"] = df.surf == '9'

var_ecartees.append(("surf", "Dichotomisation"))
log_action(f"Dichotomisation du l'état de la surface : sèche, mouillée, glissante (3 à 9)")

##############################################################################
# Dichotomisation de la vitesse maximale autorisée
##############################################################################
# la vma (vitesse maximale autorisée) est codée par un flottant
# Il y a des valeurs abberrantes et codées avec des points décimaux
# Nous les convertissons en entiers
vma_int = df.vma.astype(int)
df["vma_30m"] = vma_int.isin([10, 20, 30])
df["vma_40"]  = vma_int == 40
df["vma_50"]  = vma_int == 50
df["vma_60"]  = vma_int == 60
df["vma_70"]  = vma_int == 70
df["vma_80"]  = vma_int == 80
df["vma_90"]  = vma_int == 90
df["vma_110"] = vma_int == 110
df["vma_130"] = vma_int == 130

var_ecartees.append(("vma", "Dichotomisation"))
log_action(f"Dichotomisation de la vitesse maximale autorisée avec regroupement")

##############################################################################
# agg : En ou hors agglomération
##############################################################################
# modalités 1 et 2, sans valueurs nulles (-1 ou na)
df.agg_agg = df.agg == '1'
var_ecartees.append(("agg", "Dichotomisation"))
log_action("Dichotomisation en ou hors agglomération (agg), 1 agglomération, 0 hors agglomoration")

##############################################################################
# Dichotomisations plus simples
##############################################################################

# actp : action du piéton
dummies = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B']
dichotomisation(df, "actp", var_ecartees, desc_vars, dummies = dummies)
log_action("Dichotomisation de l'action du piéton (actp), modalité -1 0 et B exclues")

# atm : Conditions atmosphériques
dummies = ['1', '2', '3', '4', '5', '6', '7', '8']
dichotomisation(df, "atm", var_ecartees, desc_vars, dummies = dummies)
log_action("Dichotomisation des cond. atmosphériques (atm), modalité -1 et 9 exclues")

# catr : Catégorie de route
dummies = ['1', '2', '3', '4', '5', '6', '7']
dichotomisation(df, "catr", var_ecartees, desc_vars, dummies = dummies)
log_action("Dichotomisation de la catégorie de route (catr)), modalité -1 et 9 exclues")

# catu : Catégorie d'usager
dummies = ['1', '2', '3']
dichotomisation(df, "catu", var_ecartees, desc_vars, dummies = dummies)
log_action("Dichotomisation de la catégorie d'usager (catu), modalité -1 exclue")

# catv : Catégorie de véhicule
dichotomisation(df, "catv", var_ecartees, desc_vars, dummies = None, mod_ecartees=[0, -1])
log_action("Dichotomisation de la catégorie de véhicule (catv), modalité -1 et 0 exclues")

# choc : Point de choc initial
dichotomisation(df, "choc", var_ecartees, desc_vars, dummies = None, mod_ecartees=['0', '-1'])
log_action("Dichotomisation du point de choc initial (choc), modalité -1 et 0 exclues")

# circ : Circulation
dichotomisation(df, "circ", var_ecartees, desc_vars, dummies = None, mod_ecartees=['-1'])
log_action("Dichotomisation du régime de circulation (circ), modalité -1 exclue")

# col : Type de collision
dichotomisation(df, "col", var_ecartees, desc_vars, dummies = None, mod_ecartees=['-1'])
log_action("Dichotomisation du type de collision (col), modalité -1 exclue")

# etatp: Piéton seul
dichotomisation(df, "etatp", var_ecartees, desc_vars, dummies = ['1', '2', '3'])
log_action("Dichotomisation de (etatp), modalité -1 exclue")

# infra : Aménagement - infrastructure
dichotomisation(df, "infra", var_ecartees, desc_vars, dummies = None, mod_ecartees=['-1', '0'])
log_action("Dichotomisation de Aménagement - infrastructure (infra), modalités -1 et 0 exclues")

# int : type d'intersection
dichotomisation(df, "int", var_ecartees, desc_vars, dummies = None, mod_ecartees=['-1'])
log_action("Dichotomisation du type d'intersection (int), modalité -1  exclues")

# jsem : Jour de la semaine
dichotomisation(df, "jsem", var_ecartees, desc_vars, dummies = None, mod_ecartees=None)
log_action("Dichotomisation du jour de la semaine (jsem), toutes modalité ")

# locp : Localisation du piéton
dichotomisation(df, "locp", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1', '0'])
log_action("Dichotomisation de la localisation du piéton (locp), modalités -1, 0(non renseigné) exclues")

# lum : Lumière modalité
dichotomisation(df, "lum", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1'])
log_action("Dichotomisation des conditions lumineuses (lum), modalité -1 exclue")

# manv : Manœuvre
dichotomisation(df, "manv", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1', '0'])
log_action("Dichotomisation de la manœuvre (manv), modalité -1 et 0 exclues")

# mois : Mois
dichotomisation(df, "mois", var_ecartees, desc_vars, dummies = None, mod_ecartees = None)
log_action("Dichotomisation du mois (mois), toutes modalités")

# motor : Motorisation
dichotomisation(df, "motor", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1', '0'])
log_action("Dichotomisation de la motorisation (motor), modalité -1 et 0 exclues")

# obs : Obstacle fixe heurté
dichotomisation(df, "obs", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1', '0'])
log_action("Dichotomisation de l'obstacle fixe heurté (obs), modalité -1 et 0 exclues")

# obsm : Obstacle mobile heurté
dichotomisation(df, "obsm", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1', '0'])
log_action("Dichotomisation de l'obstacle mobile heurté (obsm), modalité -1 et 0 exclues")

# place : Place de l'usager dans le véhicule
dichotomisation(df, "place", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1'])
log_action("Dichotomisation de la place dans le véhicule (place), modalité -1 exclue")

# plan : Tracé en plan
dichotomisation(df, "plan", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1'])
log_action("Dichotomisation du tracé en plan (plan), modalité -1 exclue")

# prof : Déclivité
dichotomisation(df, "prof", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1'])
log_action("Dichotomisation de la déclivité (prof), modalité -1 exclue")

# senc : sens de circulation
dichotomisation(df, "senc", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1'])
log_action("Dichotomisation du sens de circulation (senc), modalité -1, 0 et 3 exclues")

# situ : Situation de l'accident
dichotomisation(df, "situ", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1'])
log_action("Dichotomisation de la situation de l'accident (situ), modalité -1 exclue")

# trajet : Motif du trajet
dichotomisation(df, "trajet", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1'])
log_action("Dichotomisation du motif du trajet (trajet), modalité -1 exclue")

# vosp : Présence d'une voie réservée
dichotomisation(df, "vosp", var_ecartees, desc_vars, dummies = None, mod_ecartees = ['-1'])
log_action("Dichotomisation de la présence d'une voie réservée (vosp), modalités -1 exclue")

 - Dichotomisation des champs secu1, secu2 et secu3
 - Dichotomisation de l'âge
 - Dichotomisation de l'heure
 - Dichotomisation du sexe
 - Dichotomisation de la gravité
 - Dichotomisation du nombre de voies avec regroupement 1 à 4 puis 5 et plus
 - Dichotomisation du l'état de la surface : sèche, mouillée, glissante (3 à 9)
 - Dichotomisation de la vitesse maximale autorisée avec regroupement
 - Dichotomisation en ou hors agglomération (agg), 1 agglomération, 0 hors agglomoration

 - Dichotomisation de l'action du piéton (actp), modalité -1 0 et B exclues

 - Dichotomisation des cond. atmosphériques (atm), modalité -1 et 9 exclues

 - Dichotomisation de la catégorie de route (catr)), modalité -1 et 9 exclues

 - Dichotomisation de la catégorie d'usager (catu), modalité -1 exclue



C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c
C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c
C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

 - Dichotomisation de la catégorie de véhicule (catv), modalité -1 et 0 exclues



C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c
C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c
C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

 - Dichotomisation du point de choc initial (choc), modalité -1 et 0 exclues



...

C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c
C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c
C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

 - Dichotomisation du motif du trajet (trajet), modalité -1 exclue



C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c
C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c


 - Dichotomisation de la présence d'une voie réservée (vosp), modalités -1 exclue


C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c
C:\Users\erika\AppData\Local\Temp\ipykernel_18872\747903453.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[column] == c


In [16]:
print ("Matin      : ", df.hr_matin.sum())
print ("Midi       : ", df.hr_midi.sum())
print ("après-midi : ", df.hr_am.sum())
print ("soir       : ", df.hr_soir.sum())
print ("Nuit       : ", df.hr_nuit.sum())

Matin      :  128204
Midi       :  51051
après-midi :  141102
soir       :  95798
Nuit       :  78027


In [17]:
print ("Enfants  : ", df.age_enfant.sum())
print ("jeunes   : ", df.age_jeune.sum())
print ("adultes  : ", df.age_adulte.sum())
print ("3ème âge : ", df.age_3age.sum())

Enfants  :  30341
jeunes   :  118147
adultes  :  287571
3ème âge :  52182


# Suppression de variables
Nous supprimons des variables pour les raisons suivantes :
  - Codage douteux, valeurs trop dispersées ;
  - Remplacées par d'autres variables (calculées) ;
  - variables dichotomisées ;
  - Index servant aux jointures et n'expliquant rien.

In [19]:
##############################################################################
# Ajout à la liste des variables trop dispersées ou douteuses pour suppression
##############################################################################
var_ecartees.append(("adr",     "Dispersion trop importante"))
var_ecartees.append(("com",     "Dispersion trop importante"))
var_ecartees.append(("dep",     "Dispersion trop importante"))
var_ecartees.append(("larrout", "Valeurs douteuses"))
var_ecartees.append(("lartpc",  "Valeurs douteuses"))
var_ecartees.append(("lat",     "Valeurs douteuses"))
var_ecartees.append(("long",    "Valeurs douteuses"))
var_ecartees.append(("occutc",  "trop de nuls"))
var_ecartees.append(("pr",      "Dispersion"))
var_ecartees.append(("pr1",     "Dispersion"))
var_ecartees.append(("v1",      "Dispersion"))
var_ecartees.append(("v2",      "Dispersion"))
var_ecartees.append(("voie",    "Dispersion"))

##############################################################################
# Ajout à la liste des variables remplacées (calculées) pour suppression
##############################################################################
var_ecartees.append(("an",      "Utilisée pour déterminer les jours fériés et l'âge puis supprimée"))
var_ecartees.append(("an_nais", "Utilisée avec 'an' pour calculer l'âge puis supprimée"))
var_ecartees.append(("jour",    "Utilisée pour déterminer si le jour est férié puis suppimée"))

##############################################################################
# Ajout à la liste des variables dichotomisées pour suppression
##############################################################################
# Les noms des variables ont déjà été ajoutées à var_ecartees
# par la fonction dichotomisation.

##############################################################################
# Ajout à la liste des variables d'index pour suppression
##############################################################################

var_ecartees.append(("Num_Acc",      "Variable d'index"))
var_ecartees.append(("id_usager",    "Variable d'index"))
var_ecartees.append(("id_vehicule",  "Variable d'index"))
var_ecartees.append(("num_veh",      "Variable d'index"))

variables_a_supprimer = [ve[0] for ve in var_ecartees]

# Lors de la mise au point de ce notebook la liste des variables à écarter
# est complétée avec des valeurs déjà présentes, drop est alors perturbée
# et ne supprime pas les variables. Il faut supprimer les doublons.
# C'est fait en convertissant la liste en ensmblme (set)
# puis en la recovertissant en liste.
variables_a_supprimer = set(variables_a_supprimer)
variables_a_supprimer = list(variables_a_supprimer)

print ("Variables à supprimer : ", variables_a_supprimer)
df = df.drop (columns = variables_a_supprimer, axis = 1)

print(f"Il reste les {df.shape[1]} colonnes suivantes :", end=" ")
print("[" + ", ".join(f"'{col}'" for col in df.columns) + "]")

print ("Valeurs nulles :\n", df.isnull().sum())

Variables à supprimer :  ['larrout', 'lat', 'jour', 'lum', 'catr', 'com', 'vma', 'prof', 'place', 'etatp', 'catu', 'actp', 'secu2', 'id_usager', 'senc', 'nbv', 'occutc', 'hrmn', 'pr1', 'num_veh', 'int', 'locp', 'secu3', 'mois', 'circ', 'Num_Acc', 'secu1', 'col', 'an', 'atm', 'motor', 'agg', 'obs', 'plan', 'situ', 'dep', 'age', 'manv', 'lartpc', 'trajet', 'an_nais', 'vosp', 'choc', 'long', 'jsem', 'voie', 'catv', 'adr', 'sexe', 'pr', 'v1', 'id_vehicule', 'v2', 'surf', 'obsm', 'infra', 'grav']
Il reste les 265 colonnes suivantes : ['ferie', 'secu_ceinture', 'secu_casque', 'secu_dispenfant', 'secu_gilet', 'secu_airbag23RM', 'secu_gants', 'age_enfant', 'age_jeune', 'age_adulte', 'age_3age', 'hr_matin', 'hr_midi', 'hr_am', 'hr_soir', 'hr_nuit', 'sexe_m', 'sexe_f', 'grav_grave', 'nbv_1', 'nbv_2', 'nbv_3', 'nbv_4', 'nbv_plus', 'surf_norm', 'surf_mouil', 'surf_gliss', 'surf_autre', 'vma_30m', 'vma_40', 'vma_50', 'vma_60', 'vma_70', 'vma_80', 'vma_90', 'vma_110', 'vma_130', 'actp_1', 'actp_2', 

# Suppression de doublons

In [21]:
avant_supp = df.shape[0]
df = df.drop_duplicates()
apres_supp = df.shape[0]
print ("Suppression d'observations :")
print (f"  avant suppression {avant_supp:7d}")
print (f"  nous supprimons   {avant_supp - apres_supp:7d}")
print (f"  après suppression {apres_supp:7d}")

Suppression d'observations :
  avant suppression  494182
  nous supprimons      3534
  après suppression  490648


# Équilibrage ou réduction de dimension
La répartition des modalités de la variable cible est très déséquilibrée
Notre objectif est de prédire les conditions des accidents ayant ayant entraîné une hospitalisation de plus de 24h ou la mort.


In [23]:
##############################################################################
# Équilibrage des modalités de la variable cible
##############################################################################

print ("Répartition avant réduction :")
print (df.value_counts("grav_grave"))
print (f"total : {df.shape[0]:6d}")
X = df.drop("grav_grave", axis = 1)
y = df.grav_grave
rus = RandomUnderSampler(random_state = 8421)
X, y = rus.fit_resample(X, y)
df = pd.concat([X, y], axis = 1)
print ("Répartition après réduction :")
print(df.value_counts("grav_grave"))
print (f"total :{df.shape[0]:6d}")

Répartition avant réduction :
grav_grave
False    401827
True      88821
Name: count, dtype: int64
total : 490648
Répartition après réduction :
grav_grave
False    88821
True     88821
Name: count, dtype: int64
total :177642


In [24]:
df.grav_grave.value_counts()

grav_grave
False    401827
True      88821
Name: count, dtype: int64

In [25]:
nb_obs = df.shape[0]
for col in df.columns:
    print(f"{col:15s}   {df[col].sum():6d}  {100.*df[col].sum()/nb_obs:10.7f}%")

ferie              68406  13.9419706%
secu_ceinture     290435  59.1941677%
secu_casque        91891  18.7284978%
secu_dispenfant     3686   0.7512514%
secu_gilet          5186   1.0569696%
secu_airbag23RM     6696   1.3647258%
secu_gants         46148   9.4055209%
age_enfant         29816   6.0768616%
age_jeune         117428  23.9332475%
age_adulte        285414  58.1708272%
age_3age           52058  10.6100504%
hr_matin          127189  25.9226574%
hr_midi            50715  10.3363307%
hr_am             140033  28.5404200%
hr_soir            95167  19.3961863%
hr_nuit            77544  15.8044056%
sexe_m            331579  67.5798128%
sexe_f            153272  31.2386884%
grav_grave         88821  18.1027947%
nbv_1              46439   9.4648302%
nbv_2             292431  59.6009767%
nbv_3              40968   8.3497742%
nbv_4              62571  12.7527270%
nbv_plus           32164   6.5554124%
surf_norm         397415  80.9979863%
surf_mouil         86345  17.5981559%
surf_gliss  

In [26]:
print ("Actions réalisées :")
for a in actions:
    print (a)

Actions réalisées :
Jointure usagers  <---  caracteristiques
Jointure (usagers et caracteristiques)  <---  lieux
Jointure (usagers, caracteristiques et lieux)  <---  vehicules
Suppression de 0 observations dont la gravité est inconnue (codée -1)
Création de la variable jsem : jour de la semaine
Création de la variable ferie : jour férié, dimanches et autres fêtes
Création de la variable age : différence entre l'année de l'accident et l'année de naissance
Dichotomisation des champs secu1, secu2 et secu3
Dichotomisation de l'âge
Dichotomisation de l'heure
Dichotomisation du sexe
Dichotomisation de la gravité
Dichotomisation du nombre de voies avec regroupement 1 à 4 puis 5 et plus
Dichotomisation du l'état de la surface : sèche, mouillée, glissante (3 à 9)
Dichotomisation de la vitesse maximale autorisée avec regroupement
Dichotomisation en ou hors agglomération (agg), 1 agglomération, 0 hors agglomoration
Dichotomisation de l'action du piéton (actp), modalité -1 0 et B exclues
Dichotomi

In [27]:
##############################################################################
# Enregistrements :
#   - Le jeu de données préparé pour la modélisation ;
#   - La description des variables mise à jour.
##############################################################################

df.to_csv(rep_dst + '/' + "data.csv", sep = '\t', index=False, encoding='utf-8')

with open("./desc_vars.json", 'w', encoding='utf-8') as fichier:
    json.dump(desc_vars, fichier, ensure_ascii=True, indent=True)